In [27]:
import itertools
import numpy as np
import plotly.express as px

import plotly.graph_objects as go

This notebook describes a few subtleties within the theory behind the problem. My criteria for what goes here was basically just what confused me personally, so sorry if some of this is dumb!

# Vectors in crystals
An NaI crystal follows an "FCC" (face-centred cubic) crystal structure: a cube where each face has a cell in the middle. A cell therefore has 8 points from the cube vertices and 6 from its faces, making 14 points per cell. The length of each cube is the _lattice constant_ $a$. The code below creates a unit FCC cell with $a=1$.

In [24]:
def lattice_3d_scratch(N_x, N_y, N_z):
    '''Creates a 2d array of 3-vectors under the FCC lattice structure. Dimension will be (L,3) where L is the 
    number of total points. Columns [0,1,2] are [x,y,z]
    Inputs are the number of _cells_ in the x/y/z direction. This is not the number of points'''
    full_grid = np.array(tuple(itertools.product(np.arange(0,N_x+0.5,0.5),
                                                 np.arange(0,N_y+0.5,0.5),
                                                 np.arange(0,N_z+0.5,0.5))))
    # Points on the vertices of cubes will have an integer position, while other points will have 
    # half-integer positions. An FCC crystal only has points with 0 or 2 non-integer points.
    return np.array(list(filter(lambda x: sum(x%1)%1 == 0, full_grid)))

lattice = lattice_3d_scratch(1,1,1)


In [25]:
fig = px.scatter_3d(x = lattice[:,0],
                    y = lattice[:,1],
                    z = lattice[:,2])

fig.update_traces(marker_size = 5)
fig.show()

This set of points is called the _Bravais lattice_, and is defined by taking integer linear combinations of the basis:
$$\vec{a}_1 = \frac{a}{2}[0,1,1]$$
$$\vec{a}_2 = \frac{a}{2}[1,0,1]$$
$$\vec{a}_3 = \frac{a}{2}[1,1,0]$$
Where square brackets are vectors in the cartesian xyz basis. Each vector points from the origin to one of the face-centred cells: $\vec{a}_1$ in the y-z plane, $\vec{a}_2$ in the x-z plane, and $\vec{a}_3$ in the x-y plane. **This order is different to how Dent et.al defines it** (they swap 1 and 2), but this order is more intuitive to me and ultimately doesn't affect any results. This is the _Bravais basis_, and defines all points for a single FCC crystal.

It is important to note that in multi-atomic crystals, there are several crystal lattices. In NaI, both Na and I have an FCC structure but are offset by the _first primitive_ vector: $$\vec{\alpha}_1 = \frac{a}{2}[1,1,1]$$
$\vec{\alpha}_1$ would be a point in the very middle of an FCC cell--and therefore wouldn't be part of the cell itself. If we define the Na points to start at the origin, then the I cells are any integer linear combination of the _Bravais_ basis vectors plus the _primitive_ basis vector.

In [37]:
Na_lattice = lattice_3d_scratch(1,1,1)
I_lattice  = Na_lattice + np.array([1/2,1/2,1/2])


fig = go.Figure()

scatter = fig.add_trace(go.Scatter3d(
    x=Na_lattice[:,0],
    y=Na_lattice[:,1],
    z=Na_lattice[:,2],
    mode='markers',
    name = "Na",
    marker = dict(
        size=5,
        color='blue'
    )
))

scatter_io = fig.add_trace(go.Scatter3d(
    x=I_lattice[:,0],
    y=I_lattice[:,1],
    z=I_lattice[:,2],
    name = "I",
    mode='markers',
    marker = dict(
        size=5,
        color='red'
    )
))

fig.show()

For some nomenclature, Na and I are the two "species" of lattices, which will be summed over in further calculations.

With the Bravais basis (or real-space basis) defined, the _reciprocal space_ (the Fourier-space) basis is defined as:
$$\vec{b}_1 = \frac{2\pi}{V} \left(\vec{a}_2\times \vec{a}_3\right)$$
$$\vec{b}_2 = \frac{2\pi}{V} \left(\vec{a}_3\times \vec{a}_1\right)$$
$$\vec{b}_3 = \frac{2\pi}{V} \left(\vec{a}_1\times \vec{a}_2\right)$$

$$V = |\vec{a}_1 \cdot (\vec{a}_2 \times \vec{a}_3)| = \frac{a^3}{4}$$
The $\vec{b}_i$ basis set is normal to each corresponding Bravais basis vector $\vec{a}_i$, and the scaling factor of $V$ represents the volume of a Bravais basis paralellepiped. This means any vector in the _reciprocal space_ (the span of the _reciprocal basis_) has units of inverse length. 

We can cast the reciprocal basis into a more simple form by using the _hkl_ basis, which writes a reciprocal space vector $\vec{G}$ in terms of an integer space:
$$\vec{G} = \frac{2\pi}{a}(h,k,l)$$
Using the fact that each component in $\vec{a}_i\times\vec{a}_j$ scales with volume nicely to create $\frac{2\pi}{a}$.
This form becomes useful when complex exponentials are involved. The (h,k,l) here represents integer valued cartesian vectors; i.e., if we're looking at the vertex of an FCC cell furthest from the origin we have
$$\vec{G} = \frac{2\pi}{a}(1,1,1)$$



# The crystal structure factor (important)

The equation we're working towards is:
$$\frac{dN}{dt} = \pi g_{a\gamma}^2 (\hbar c)^3 \frac{V}{V_{cell}^2}\sum_{\vec{G}} \frac{d\Phi_a}{dE_a} \frac{|F(\vec{G}) S(\vec{G})|^2}{|\vec{G}|^2} \sin^2\left[2\theta(\vec{k},\vec{G})\right] \mathcal{W}\left(E_a(\vec{G}),\Delta \right)$$
Most of this is well defined within the Dent et.al paper, except $F$ and $S$, which are defined differently in different places. The key thing to note is that this equation deceptively assumes a monatomic crystal, when a multi-atom crystal like NaI actually sums over different species.

F is the _atomic form factor_, explained in the Dent et.al paper in equation (3):
$$F_A(q) = \frac{Zr_0^2 q^2}{1+r_0^2 q^2}$$
where $A$ is the atomic species (Na or I).

$S$ is the _crystal structure factor_. This is given by:
$$S(\vec{G}) =\sum_{s~\in~\text{species}} \sum_{i=1}^3 e^{i \vec{G}\cdot(\vec{a}_i + \vec{\alpha}_s)}$$
Where $\vec{a}_i$ is the $i$ th Bravais basis vector (defined above) and $ \vec{\alpha}_s $ is the primitive vector for species $s$. We can set $\vec{\alpha}_{Na}$ = 0 and $\alpha_{I} = \frac{a}{2}[1,1,1]$ for Na and I, but thankfully swapping these two makes no difference. Computing this sum for a fixed (h,k,l) and factorising, we get:
$$S(\vec{G}) = (1+e^{i\pi(h+k)} + e^{i\pi (k+l)}+ e^{i\pi (h+l)})(1+e^{i\pi(h+k+l)})$$
The first term is $4$ when h,k,l share a parity and 0 otherwise. The second term is 2 when h+k+l is even and 0 otherwise. Combining the two conditions, $S=8$ when h,k,l are all even and 0 otherwise.

Combining this with the form factor, we require h,k,l all even in the sum, and the $F(\vec{G})S(\vec{G})$ term is
$$\left|8(F_{Na}(|\vec{G}|) + F_{I}(|\vec{G}|)) \right|^2 = 64\left( F_{Na}(|\vec{G}|) + F_{I}(|\vec{G}|)\right)^2$$
Where we use the form for $F_A(q)$ quoted above.